In [69]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import configparser

isLiveApi = 0
epic = 'CS.D.EURUSD.MINI.IP'
resolution = 'DAY'
# Ideal: '2006-01-02T00:00:00' -> '2024-03-29T00:00:00'
start_date = datetime.strptime('2006-01-01T00:00:00', '%Y-%m-%dT%H:%M:%S')
end_date = datetime.strptime('2024-05-15T00:00:00', '%Y-%m-%dT%H:%M:%S')
delta = timedelta(days=20) # historical data can fetch one API call
# List to hold all data frames
data_frames = []

# Read credentials from the configuration file
config = configparser.ConfigParser()
config.read('ig.cfg')

# Function to generate date ranges in 20-day increments
def date_range(start_date, end_date, delta):
    current_date = start_date
    while current_date < end_date:
        yield current_date, min(current_date + delta, end_date)
        current_date += delta
        
if isLiveApi == 0:
    API_KEY = config.get('demo', 'api_key')
    USERNAME = config.get('demo', 'username')
    PASSWORD = config.get('demo', 'password')
    BASE_URL = config.get('demo', 'base_url')
    ACC_ID = config.get('demo', 'acc_number')
else:
    API_KEY = config.get('live', 'api_key')
    USERNAME = config.get('live', 'username')
    PASSWORD = config.get('live', 'password')
    BASE_URL = config.get('live', 'base_url')
    ACC_ID = config.get('live', 'acc_number')

# IG API URLs for demo account
LOGIN_URL = BASE_URL + 'session'
MARKET_URL = BASE_URL + 'markets'
HISTORICAL_DATA_URL = BASE_URL + 'prices/' + epic
# HISTORICAL_DATA_URL = BASE_URL + 'prices/{epic}/{resolution}/{start_date}/{end_date}'

# Authenticate and obtain access token
auth_response = requests.post(LOGIN_URL, json={
    'identifier': USERNAME,
    'password': PASSWORD
}, headers={
    'Content-Type': 'application/json; charset=UTF-8',
    'Accept': 'application/json; charset=UTF-8',
    'X-IG-API-KEY': API_KEY,
    'Version': '3',
    'IG-ACCOUNT-ID': ACC_ID
    }
)

# Extract the access token from the response
access_token = auth_response.json()['oauthToken']['access_token']
token_type = auth_response.json()['oauthToken']['token_type']

# Loop through each date range and fetch data
for start, end in date_range(start_date, end_date, delta):
    start_str = start.strftime('%Y-%m-%dT%H:%M:%S')
    end_str = end.strftime('%Y-%m-%dT%H:%M:%S')

    # Request historical price data for the specified instrument
    history_response = requests.get(HISTORICAL_DATA_URL,
        headers={
            'Content-Type': 'application/json; charset=UTF-8',
            'Accept': 'application/json; charset=UTF-8',
            'Authorization': f"{token_type} {access_token}",
            'Version': '3',
            'X-IG-API-KEY': API_KEY,
            'IG-ACCOUNT-ID': ACC_ID
        },
        params={
            'resolution': resolution,
            'from': start_str,
            'to': end_str,
        }
    )

    # Check for successful response
    if history_response.status_code == 200:
        # Convert JSON response to pandas DataFrame and append to list
        data = history_response.json()
        df = pd.DataFrame(data['prices'])
        data_frames.append(df)
    else:
        print(f"Failed to fetch data from {start_str} to {end_str}. Status code: {history_response.status_code}")
        break

# Concatenate all data frames into one
if data_frames:
    all_data = pd.concat(data_frames, ignore_index=True)
    # Write the concatenated data frame to a CSV file
    csv_file_name = f'./getdata/ig/{epic}_{resolution}_{start_date.strftime("%Y-%m-%d")}_{end.strftime("%Y-%m-%d")}.csv'
    all_data.to_csv(csv_file_name, index=False)
    print(f"Data has been successfully written to {csv_file_name}")
else:
    print("No data was fetched.")

# # Request historical price data for the specified instrument
# # Ideal: '2006-01-02T00:00:00' -> '2024-03-29T00:00:00'
# history_response = requests.get(f"{HISTORICAL_DATA_URL}",
#     headers= {
#         'Content-Type': 'application/json; charset=UTF-8',
#         'Accept': 'application/json; charset=UTF-8',
#         'Authorization': f"{token_type} {access_token}",
#         'Version': '3',
#         'X-IG-API-KEY': API_KEY,
#         'IG-ACCOUNT-ID': ACC_ID
#     }, 
#     params={
#         'resolution': resolution,
#         'from': start,
#         'to': end,
#     }
# )
# # Convert JSON response to pandas DataFrame
# # data = history_response.json()
# df = pd.DataFrame(history_response.json()['prices'])

# # Write DataFrame to CSV file
# df.to_csv('./getdata/ig/'+ epic + '_' + resolution + '_' + start_date + '_' + end_date + '.csv', index=False)
# Check status code
# if history_response.status_code == 200:
#     print("Historical price data retrieved successfully!")
#     print(history_response.json())  # Print response body
# else:
#     # Print the URL of the GET request
#     print("Request URL:", history_response.request.url)
#     print(f"Error: {history_response.status_code} - {history_response.text}")  # Print error message
# # Print the historical price data
# print(history_response)
# # Retrieve instrument ID for a specific market
# market_response = requests.get(MARKET_URL, headers={
#     'Authorization': f'Bearer {access_token}',
#     'X-IG-API-KEY': API_KEY
# })
# print(f"market_response = {market_response}")

# # Headers for authentication
# headers = {
#     'X-IG-API-KEY': API_KEY,
#     'Content-Type': 'application/json; charset=UTF-8',
#     'Accept': 'application/json; charset=UTF-8',
# }

# # Authenticate and get session tokens
# auth_data = {
#     'identifier': USERNAME,
#     'password': PASSWORD
# }
# auth_response = requests.post(LOGIN_URL, json=auth_data, headers=headers)
# # Pretty print the response headers
# print("Response Headers:")
# print(json.dumps(dict(auth_response.headers), indent=4))

# # Extract tokens
# CST = auth_response.headers['CST']
# X_SECURITY_TOKEN = auth_response.headers['X-SECURITY-TOKEN']

# # Update headers with tokens for subsequent requests
# headers['CST'] = CST
# headers['X-SECURITY-TOKEN'] = X_SECURITY_TOKEN

# # Function to get historical data
# def get_historical_data(epic, resolution, start_date, end_date):
#     url = HISTORICAL_DATA_URL.format(epic=epic, resolution=resolution, start_date=start_date, end_date=end_date)
#     print(f"url = {url}")
#     print(f"headers = {headers}")
#     response = requests.get(url, headers=headers)
#     # print("Response Headers:")
#     # print(json.dumps(dict(response.headers), indent=4))
#     # print(f"Response Data: {response.text}")
#     data = response.json()
#     print(data)
    
#     # Convert the data to a DataFrame
#     prices = data['prices']
#     df = pd.DataFrame(prices)
#     return df

# # Define the instrument epic, resolution, and date range
# epic = 'CS.D.EURUSD.MINI.IP'  # Example epic for EUR/USD
# resolution = 'DAY'  # Resolution: MINUTE, HOUR, DAY, etc.
# # start_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%dT%H:%M:%S')  # 7 days ago
# # end_date = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')  # Now
# start_date = '2023-05-15T00%3A00%3A00'
# end_date = '2024-05-15T00%3A00%3A00'

# # Get historical data
# df = get_historical_data(epic, resolution, start_date, end_date)
# # print(df)


Data has been successfully written to ./getdata/ig/CS.D.EURUSD.MINI.IP_DAY_2006-01-01_2006-01-02.csv
